In [1]:
import datetime as dt
import pandas as pd

from de.processor.source.dart import DART_FUNDAMENTAL_PROCESSOR, DART_INFO_PROCESSOR
from de.processor.source.krx import KRX_INFO_PROCESSOR
from de.processor.source.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

In [2]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [3]:
# dart_info
dart_info_df = pd.read_csv("./de/data/dart_info_df.csv", index_col=0)
dart_info_processor = DART_INFO_PROCESSOR(dart_info_df)

In [4]:
# krx_info
krx_info_df = pd.read_csv("./de/data/krx_info_df.csv", index_col=0)
krx_info_processor = KRX_INFO_PROCESSOR(krx_info_df)

In [5]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [6]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [7]:
class STOCK_INFO_PROCESOR:
    def __init__(
        self, fdr_info_processor, krx_info_processor, dart_info_processor, dart_fundamental_processor, ohlcv_df
    ):
        self.fdr_info_processor = fdr_info_processor
        self.krx_info_processor = krx_info_processor
        self.dart_info_processor = dart_info_processor
        self.dart_fundamental_processor = dart_fundamental_processor
        self.ohlcv_df = ohlcv_df

    @staticmethod
    def append_StockName(df, fdr_info_processor):
        df["StockName"] = df["StockCode"].map(fdr_info_processor.get_StockName_dict())
        return df

    @staticmethod
    def append_MarketSector(df, krx_info_processor):
        df["MarketSector"] = df["StockCode"].map(krx_info_processor.get_MarketSector_dict())
        return df

    @staticmethod
    def append_Product(df, dart_info_processor):
        df["Product"] = df["StockCode"].map(dart_info_processor.get_Product_dict())
        return df

    @staticmethod
    def append_bps(df, dart_fundamental_processor, fdr_info_processor):
        df["BPS"] = (
            df["StockCode"].map(dart_fundamental_processor.get_TotalAssets_dict())
            / df["StockCode"].map(fdr_info_processor.get_Shares_dict())
        ).astype(int)
        return df

    @staticmethod
    def append_eps(df, dart_fundamental_processor, fdr_info_processor):
        df["EPS"] = (
            df["StockCode"].map(dart_fundamental_processor.get_NetProfit_dict())
            / df["StockCode"].map(fdr_info_processor.get_Shares_dict())
        ).astype(int)
        return df

    @staticmethod
    def append_price(df, ohlcv_df):
        price_dict = ohlcv_df[ohlcv_df["Date"] == ohlcv_df["Date"].max()].set_index("StockCode")["Close"].to_dict()
        df["Price"] = df["StockCode"].map(price_dict)
        return df

    def get_stock_info_df(self, stockcodes):
        stock_info_df = pd.DataFrame(stockcodes, columns=["StockCode"])
        stock_info_df = self.append_StockName(stock_info_df, self.fdr_info_processor)
        stock_info_df = self.append_MarketSector(stock_info_df, self.krx_info_processor)
        stock_info_df = self.append_Product(stock_info_df, self.dart_info_processor)
        stock_info_df = self.append_eps(stock_info_df, self.dart_fundamental_processor, self.fdr_info_processor)
        stock_info_df = self.append_bps(stock_info_df, self.dart_fundamental_processor, self.fdr_info_processor)
        stock_info_df = self.append_price(stock_info_df, self.ohlcv_df)
        return stock_info_df

In [8]:
stock_info_procesor = STOCK_INFO_PROCESOR(
    fdr_info_processor, krx_info_processor, dart_info_processor, dart_fundamental_processor, ohlcv_df
)

In [9]:
f_ma_stockcodes = {'047050', '063170', '066970', '079810'}
f_ma_stock_info_df = stock_info_procesor.get_stock_info_df(stockcodes=f_ma_stockcodes)
f_ma_stock_info_df

,StockCode,StockName,MarketSector,Product,EPS,BPS,Price
0,066970,엘앤에프,KOSDAQ_일반전기전자,2차전지 양극활물질,1529,108461,176300
1,063170,서울옥션,KOSDAQ_유통,미술품경매,-119,10797,12930
2,047050,포스코인터내셔널,KOSPI_유통업,"무역(철강,화학,자동차부품),수출주선,대행,알선,가공원단,폴리우레탄,자동차시트 제조",992,72989,57200
3,079810,디이엔티,KOSDAQ_기계·장비,FPD공정검사장비,30,9784,17650


In [10]:
f_lc_stockcodes = {'023960', '298040'}
f_lc_stock_info_df = stock_info_procesor.get_stock_info_df(stockcodes=f_lc_stockcodes)
f_lc_stock_info_df

,StockCode,StockName,MarketSector,Product,EPS,BPS,Price
0,298040,효성중공업,KOSPI_전기전자,-,3937,383583,210000
1,023960,에쓰씨엔지니어링,KOSPI_건설업,"기술용역(건축설계) 서비스/화학플랜트 설비,기계 제작(음식료,석유화학사업관련)/유공...",-42,2620,1848
